<a href="https://colab.research.google.com/github/EmmaLimoli/big-data-project/blob/main/Ebook_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRe

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EBooks").getOrCreate()

In [4]:
# import Sparkfiles to read the amazon link, like a csv. 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
ebooks_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"), sep="\t", header=True)

# Show DataFrame
ebooks_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [5]:
# count the number of rows

ebooks_df.count()

5101693

In [6]:
# drop na

ebooks_df.dropna()

DataFrame[marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [7]:
# count book check

ebooks_df.count()

5101693

In [8]:
# drop duplicates
ebooks_df.dropDuplicates()

DataFrame[marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [9]:
# print schema

ebooks_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
# import StructField 
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [11]:
# check columns 
ebooks_df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [13]:
# transform the dataset to fit the schema file
# create table 1: review_id_table

ebook_table_1 = ebooks_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
ebook_table_1.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RGYFDX8QXKEIR|   33605939|B007KO2MLO|     328837464| 2013-09-09|
|R13CBGTMNV9R8Z|   34058393|B005FLODDE|     764276359| 2013-09-09|
| R7DRFHC0F71O0|   39601147|B00EA3L35O|     535606445| 2013-09-09|
|R27LUKEXU3KBXQ|   17351407|B00BL3JV50|     240053004| 2013-09-09|
|R1VXTPUYMNU687|   10463387|B00CXU7U80|     931529805| 2013-09-09|
|R30DKW1GJWLPZC|   50484904|B004EWGS5G|     442453110| 2013-09-09|
|R18DPFG2FALJI9|    7145636|B00BNRJAT6|     856774152| 2013-09-09|
|R24D677N5WBW5Q|    6285538|B007FZOXJM|       5589837| 2013-09-09|
|R2FCJ9BQLSIOR3|   10278048|B00B6AK7LU|     362701357| 2013-09-09|
|R1R6K4MAKDWTXI|   16568972|B00EVMMLU0|     342745087| 2013-09-09|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [14]:
# import to_date for the year and change reivew_date to date
from pyspark.sql.functions import year, month, to_date

final_1 = ebook_table_1.withColumn('review_date', to_date(ebook_table_1.review_date, 'yyyy-MM-dd'))

In [16]:
# check schema
final_1.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: date (nullable = true)



In [18]:
# change the structure
final_df = final_1.selectExpr("cast(review_id as string) review_id", "cast(customer_id as int) customer_id", "cast(product_id as string) product_id", "cast(product_parent as int) product_parent", "cast(review_date as date) review_date")
final_df.printSchema()
final_df.show()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RGYFDX8QXKEIR|   33605939|B007KO2MLO|     328837464| 2013-09-09|
|R13CBGTMNV9R8Z|   34058393|B005FLODDE|     764276359| 2013-09-09|
| R7DRFHC0F71O0|   39601147|B00EA3L35O|     535606445| 2013-09-09|
|R27LUKEXU3KBXQ|   17351407|B00BL3JV50|     240053004| 2013-09-09|
|R1VXTPUYMNU687|   10463387|B00CXU7U80|     931529805| 2013-09-09|
|R30DKW1GJWLPZC|   50484904|B004EWGS5G|     442453110| 2013-09-09|
|R18DPFG2FALJI9|    7145636|B00BNRJAT6|     856774152| 2013-09-09|
|R24D677N5WBW5Q|    6285538|B007FZOXJM|       5589837| 2013-09-09|
|R2FCJ9BQLSIOR3|   10278048|B00B6AK7LU|   

In [20]:
# create table 2 product_if and product_title

table_2 = ebooks_df.select(["product_id", "product_title"])
table_2.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B007KO2MLO|           Big Maria|
|B005FLODDE|The Woman Who Was...|
|B00EA3L35O|Mary had A Sleepy...|
|B00BL3JV50|          Starstruck|
|B00CXU7U80|The Complete Cona...|
|B004EWGS5G|The Middle Passag...|
|B00BNRJAT6|Hide in Plain Sig...|
|B007FZOXJM|Face of Betrayal ...|
|B00B6AK7LU|Final Justice (A ...|
|B00EVMMLU0|Falling For My Hu...|
+----------+--------------------+
only showing top 10 rows



In [21]:
# change the structure types for table 2
product = table_2.selectExpr("cast(product_id as string) product_id", "cast(product_title as string) product_title")
product.printSchema()
product.show()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B007KO2MLO|           Big Maria|
|B005FLODDE|The Woman Who Was...|
|B00EA3L35O|Mary had A Sleepy...|
|B00BL3JV50|          Starstruck|
|B00CXU7U80|The Complete Cona...|
|B004EWGS5G|The Middle Passag...|
|B00BNRJAT6|Hide in Plain Sig...|
|B007FZOXJM|Face of Betrayal ...|
|B00B6AK7LU|Final Justice (A ...|
|B00EVMMLU0|Falling For My Hu...|
|B00E4W4984|  Forgetting Tabitha|
|B005A1JBB8|The Blacksmith's ...|
|B0044DELIQ|Vampire Vacation:...|
|B00DJQYATE|By the Tail (Coug...|
|B004XFYNNS|The Kingdom (A Fa...|
|B00A27Z6GO|6 Erotic Bedtime ...|
|B008EGV4BQ|The Purpose Drive...|
|B00655U3WE|Insurgent (Diverg...|
|B006NG0GXQ|Forever His: A Ti...|
|B008J4RPWA|Summerset Abbey: ...|
+----------+--------------------+
only showing top 20 rows



In [23]:
# start to create table three
customer_count = ebooks_df.select(ebooks_df["customer_id"]).show()

+-----------+
|customer_id|
+-----------+
|   33605939|
|   34058393|
|   39601147|
|   17351407|
|   10463387|
|   50484904|
|    7145636|
|    6285538|
|   10278048|
|   16568972|
|   26001763|
|   48203259|
|   45028038|
|   42554355|
|   11251156|
|   44677006|
|   34645512|
|   16137650|
|    4514126|
|   44803855|
+-----------+
only showing top 20 rows



In [25]:
# use groupby and count to figure out customer_id count
customer_df = ebooks_df.groupBy("customer_id").count()
customer_df.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    7626370|   55|
|   30071668|    1|
|   15820071|    6|
|   37504465|    3|
|   15009031|    2|
|   49561237|    1|
|   35646760|   13|
|   49448186|    1|
|   13883028|    2|
|   16952526|    1|
+-----------+-----+
only showing top 10 rows



In [26]:
# print schema
customer_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- count: long (nullable = false)



In [27]:
# change the name here of count to customer_count
customer_c = customer_df.withColumnRenamed('count', 'customer_count')
customer_c.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    7626370|            55|
|   30071668|             1|
|   15820071|             6|
|   37504465|             3|
|   15009031|             2|
|   49561237|             1|
|   35646760|            13|
|   49448186|             1|
|   13883028|             2|
|   16952526|             1|
|   53083469|             2|
|   11957493|           143|
|   22588149|             7|
|   15771684|             5|
|   34771158|             2|
|   51824967|            26|
|   12850532|             1|
|    6805465|             4|
|   19240320|             2|
|   38221752|            13|
+-----------+--------------+
only showing top 20 rows



In [31]:
# change the structure type
customer_final = customer_c.selectExpr("cast(customer_id as int) customer_id", "cast(customer_count as int) customer_count")
customer_final.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [33]:
# create table 4: vine table

vine_table = ebooks_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RGYFDX8QXKEIR|          4|            0|          0|   N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|
| R7DRFHC0F71O0|          5|            0|          0|   N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|
|R1VXTPUYMNU687|          5|            1|          2|   N|
|R30DKW1GJWLPZC|          3|            1|          2|   N|
|R18DPFG2FALJI9|          5|            0|          0|   N|
|R24D677N5WBW5Q|          5|            0|          0|   N|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|
|R1R6K4MAKDWTXI|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [34]:
# change the type structure 
final_vine = vine_table.selectExpr("cast(review_id as string) review_id", "cast(star_rating as int) star_rating", "cast(helpful_votes as int) helpful_votes", "cast(total_votes as int) total_votes", "cast(vine as string) vine")
final_vine.printSchema()
final_vine.show(10)

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RGYFDX8QXKEIR|          4|            0|          0|   N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|
| R7DRFHC0F71O0|          5|            0|          0|   N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|
|R1VXTPUYMNU687|          5|            1|          2|   N|
|R30DKW1GJWLPZC|          3|            1|          2|   N|
|R18DPFG2FALJI9|          5|            0|          0|   N|
|R24D677N5WBW5Q|          5|            0|          0|   N|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|
|R1R6K4MAKDWTXI|          4|            0|          0|   N|
+--

In [ ]:
# load the dataframe into the RDS instance
# connect to postgres

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.czptynze0rvd.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [36]:
# Write DataFrame to table 1 review_id_table in RDS

final_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to table 2 products in RDS

product.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# table 3: customers

customer_final.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# table 4 vine_table

final_vine.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)